# Demask, Tokenize và Gán Labels cho PII Dataset

Notebook này thực hiện:
1. **Demask**: Thay thế PII placeholders bằng giá trị thực từ danh sách tiếng Việt
2. **Tokenize**: Sử dụng mBERT tokenizer để tokenize text
3. **Gán Labels**: Tạo BIO labels dựa trên các entity được đánh dấu bằng XML tags


## 1. Import thư viện


In [21]:
import pandas as pd
import json
import random
import re
from transformers import AutoTokenizer
import os


## 2. Load mBERT Tokenizer


In [22]:
# Load mBERT tokenizer (ưu tiên từ model local nếu có)
model_path = "pii_mBert_case"

if os.path.exists(model_path) and os.path.exists(os.path.join(model_path, "tokenizer.json")):
    print(f"✓ Load tokenizer từ model local: {model_path}")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
else:
    print("✓ Load tokenizer từ HuggingFace: bert-base-multilingual-cased")
    tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

print(f"Tokenizer loaded! Vocabulary size: {len(tokenizer.vocab)}")


✓ Load tokenizer từ HuggingFace: bert-base-multilingual-cased
Tokenizer loaded! Vocabulary size: 119547


## 3. Load danh sách PII tiếng Việt


In [23]:
# Load PII lists từ JSON file
pii_json_path = "data/pii_entities_by_label_vietnamese_v10.json"

with open(pii_json_path, 'r', encoding='utf-8') as f:
    pii_data = json.load(f)

print(f"✓ Đã load {len(pii_data)} loại PII từ JSON")
print(f"\nCác loại PII có sẵn:")
for label, values in list(pii_data.items())[:10]:
    print(f"  - {label}: {len(values)} giá trị")


✓ Đã load 57 loại PII từ JSON

Các loại PII có sẵn:
  - PHONEIMEI: 1707 giá trị
  - JOBAREA: 47 giá trị
  - FIRSTNAME: 2823 giá trị
  - VEHICLEVIN: 819 giá trị
  - AGE: 311 giá trị
  - GENDER: 2 giá trị
  - HEIGHT: 492 giá trị
  - BUILDINGNUMBER: 906 giá trị
  - MASKEDNUMBER: 1946 giá trị
  - PASSWORD: 2352 giá trị


## 4. Load dữ liệu từ CSV


In [38]:
# Load CSV file
csv_path = r"D:\CODE\PII\data\dong10000_32000_data1.csv"

# Đọc file CSV
df = pd.read_csv(csv_path, encoding='utf-8')
# Tùy chọn: Giới hạn số dòng để xử lý (bỏ comment nếu muốn giới hạn)
# df = df.head(4000).copy()
print(f"✓ Đã load {len(df)} dòng từ CSV")
print(f"\nCác cột trong file:")
for col in df.columns:
    print(f"  - {col}")

# Kiểm tra cột target_text_vi
if 'target_text_vi' in df.columns:
    print(f"\n✓ Cột 'target_text_vi' có sẵn")
    print(f"\nVí dụ dòng đầu tiên:")
    print(df['target_text_vi'].iloc[0][:200])
else:
    print(f"\n⚠ Cột 'target_text_vi' không có trong file!")
    print(f"Các cột có sẵn: {list(df.columns)}")


✓ Đã load 24296 dòng từ CSV

Các cột trong file:
  - source_text
  - target_text
  - tokens
  - labels

⚠ Cột 'target_text_vi' không có trong file!
Các cột có sẵn: ['source_text', 'target_text', 'tokens', 'labels']


## 10. In ra từng entity của mỗi sample


In [39]:
import ast

def extract_entities_from_labels(tokens, labels):
    """
    Trích xuất các entity từ tokens và labels
    
    Args:
        tokens: List tokens
        labels: List labels theo BIO format
    
    Returns:
        List các tuple (entity_text, entity_label)
    """
    entities = []
    current_entity = []
    current_label = None
    
    for token, label in zip(tokens, labels):
        if label.startswith('B-'):
            # Bắt đầu entity mới
            if current_entity:
                # Lưu entity trước đó
                entity_text = tokenizer.convert_tokens_to_string(current_entity).strip()
                entities.append((entity_text, current_label))
            current_entity = [token]
            current_label = label[2:]  # Bỏ 'B-' prefix
        elif label.startswith('I-') and current_label == label[2:]:
            # Tiếp tục entity hiện tại
            current_entity.append(token)
        else:
            # Kết thúc entity
            if current_entity:
                entity_text = tokenizer.convert_tokens_to_string(current_entity).strip()
                entities.append((entity_text, current_label))
                current_entity = []
                current_label = None
    
    # Lưu entity cuối cùng nếu có
    if current_entity:
        entity_text = tokenizer.convert_tokens_to_string(current_entity).strip()
        entities.append((entity_text, current_label))
    
    return entities

# Load file đã xử lý
df_loaded = pd.read_csv('data/demasked_and_tokenized.csv', encoding='utf-8')

print(f"Đã load {len(df_loaded)} samples\n")
print("=" * 80)

# In ra từng entity của mỗi sample
for idx in range(len(df_loaded)):
    print(f"\n[Sample {idx + 1}]")
    print(f"Source text: {df_loaded['source_text'].iloc[idx][:150]}...")
    print(f"Target text: {df_loaded['target_text'].iloc[idx][:150]}...")
    
    try:
        # Parse tokens và labels
        tokens = ast.literal_eval(df_loaded['tokens'].iloc[idx])
        labels = ast.literal_eval(df_loaded['labels'].iloc[idx])
        
        # Trích xuất entities
        entities = extract_entities_from_labels(tokens, labels)
        
        if entities:
            print(f"\nEntities tìm thấy ({len(entities)}):")
            for i, (entity_text, entity_label) in enumerate(entities, 1):
                print(f"  {i}. [{entity_label}] {entity_text}")
        else:
            print("\n  Không có entity nào")
            
    except Exception as e:
        print(f"\n  ⚠ Lỗi khi parse: {e}")
    
    print("-" * 80)
    
    # Chỉ in 10 samples đầu tiên để demo (có thể bỏ comment để in tất cả)
    if idx >= 9:
        print(f"\n... (còn {len(df_loaded) - 10} samples khác)")
        break


Đã load 20454 samples


[Sample 1]
Source text: Học sinh đánh giá của bạn được tìm thấy trên thiết bị có số serial IMEI: 20-612819-508665-8. Tài liệu này thuộc nhiều chủ đề trao đổi trong chúng ta Đ...
Target text: Học sinh đánh giá của bạn được tìm thấy trên thiết bị có số serial IMEI: [PHONEIMEI]. Tài liệu này thuộc nhiều chủ đề trao đổi trong chúng ta [JOBAREA...

Entities tìm thấy (2):
  1. [PHONEIMEI] 20 - 612819 - 508665 - 8
  2. [JOBAREA] Điện tử
--------------------------------------------------------------------------------

[Sample 2]
Source text: Xin chào Linh, theo ghi chép của chúng tôi, giấy phép 4KHLBP0ZT3AX42032 của bạn vẫn đang được ghi nhận trong danh sách của chúng tôi để truy cập vào c...
Target text: Xin chào [FIRSTNAME], theo ghi chép của chúng tôi, giấy phép [VEHICLEVIN] của bạn vẫn đang được ghi nhận trong danh sách của chúng tôi để truy cập vào...

Entities tìm thấy (2):
  1. [FIRSTNAME] Linh
  2. [VEHICLEVIN] 4KHLBP0ZT3AX42032
---------------------------------

## 11. In tất cả entities hoặc lưu vào file (tùy chọn)


In [40]:
# Tùy chọn: In tất cả samples hoặc lưu vào file
PRINT_ALL = False  # Đặt True để in tất cả samples
SAVE_TO_FILE = True  # Đặt True để lưu vào file

if PRINT_ALL or SAVE_TO_FILE:
    all_entities_data = []
    
    for idx in range(len(df_loaded)):
        try:
            tokens = ast.literal_eval(df_loaded['tokens'].iloc[idx])
            labels = ast.literal_eval(df_loaded['labels'].iloc[idx])
            entities = extract_entities_from_labels(tokens, labels)
            
            sample_data = {
                'sample_id': idx + 1,
                'source_text': df_loaded['source_text'].iloc[idx],
                'target_text': df_loaded['target_text'].iloc[idx],
                'num_entities': len(entities),
                'entities': entities
            }
            all_entities_data.append(sample_data)
            
            if PRINT_ALL:
                print(f"\n[Sample {idx + 1}]")
                print(f"Source: {df_loaded['source_text'].iloc[idx][:100]}...")
                if entities:
                    print(f"Entities ({len(entities)}):")
                    for i, (entity_text, entity_label) in enumerate(entities, 1):
                        print(f"  {i}. [{entity_label}] {entity_text}")
                else:
                    print("  Không có entity")
                print("-" * 80)
                
        except Exception as e:
            if PRINT_ALL:
                print(f"\n[Sample {idx + 1}] ⚠ Lỗi: {e}")
    
    if SAVE_TO_FILE:
        # Lưu dưới dạng JSON
        import json
        output_json = 'data/entities_by_sample.json'
        with open(output_json, 'w', encoding='utf-8') as f:
            json.dump(all_entities_data, f, ensure_ascii=False, indent=2)
        print(f"\n✓ Đã lưu {len(all_entities_data)} samples vào: {output_json}")
        
        # Lưu dưới dạng CSV (flattened)
        entities_rows = []
        for sample in all_entities_data:
            if sample['entities']:
                for entity_text, entity_label in sample['entities']:
                    entities_rows.append({
                        'sample_id': sample['sample_id'],
                        'entity_label': entity_label,
                        'entity_text': entity_text,
                        'source_text': sample['source_text'][:200]  # Truncate để dễ đọc
                    })
            else:
                entities_rows.append({
                    'sample_id': sample['sample_id'],
                    'entity_label': None,
                    'entity_text': None,
                    'source_text': sample['source_text'][:200]
                })
        
        df_entities = pd.DataFrame(entities_rows)
        output_csv = 'data/entities_by_sample.csv'
        df_entities.to_csv(output_csv, index=False, encoding='utf-8')
        print(f"✓ Đã lưu entities vào: {output_csv}")
        print(f"  Tổng số dòng: {len(df_entities)}")
        print(f"  Số samples có entity: {df_entities['entity_label'].notna().sum()}")



✓ Đã lưu 20454 samples vào: data/entities_by_sample.json
✓ Đã lưu entities vào: data/entities_by_sample.csv
  Tổng số dòng: 57120
  Số samples có entity: 56623


## 5. Hàm Demask và Tokenize


In [41]:
def demask_and_tokenize(text, pii_data, tokenizer):
    """
    Demask text và tokenize với mBERT, đồng thời gán labels
    
    Args:
        text: Text có chứa PII placeholders như [LABEL] hoặc <LABEL>value</LABEL>
        pii_data: Dictionary chứa các danh sách PII theo label
        tokenizer: mBERT tokenizer
    
    Returns:
        source_text: Text đã demask (không có XML tags)
        target_text: Text gốc với placeholders
        tokens: List tokens từ mBERT
        labels: List labels theo BIO format
    """
    if pd.isna(text) or text == '':
        return '', '', [], []
    
    text = str(text)
    target_text = text  # Giữ nguyên text gốc
    
    # Bước 1: Demask - Thay [LABEL] bằng <LABEL>value</LABEL>
    pattern1 = r'\[([A-Z_0-9]+)\]'  # Thêm 0-9 để match IPV4, IPV6, v.v.
    def replace_label1(match):
        label = match.group(1)
        if label in pii_data and len(pii_data[label]) > 0:
            return f"<{label}>{random.choice(pii_data[label])}</{label}>"
        return match.group(0)  # Giữ nguyên nếu không tìm thấy
    
    text_with_xml = re.sub(pattern1, replace_label1, text)
    
    # Bước 2: Loại bỏ XML tags để tạo source_text (chỉ giữ giá trị)
    pattern_xml = r'<([A-Z_0-9]+)>(.*?)</\1>'  # Thêm 0-9 để match IPV4, IPV6, v.v.
    source_text = re.sub(pattern_xml, r'\2', text_with_xml)
    
    # Bước 3: Tokenize và gán labels dựa trên XML tags
    tokens = []
    labels = []
    
    # Parse text với XML tags để tách entity và non-entity
    segments = []
    last_pos = 0
    
    for match in re.finditer(pattern_xml, text_with_xml):
        # Text trước entity
        before_text = text_with_xml[last_pos:match.start()].strip()
        if before_text:
            segments.append((before_text, None))
        
        # Entity text
        label = match.group(1)
        entity_text = match.group(2).strip()
        if entity_text:
            segments.append((entity_text, label))
        
        last_pos = match.end()
    
    # Text sau entity cuối cùng
    after_text = text_with_xml[last_pos:].strip()
    if after_text:
        segments.append((after_text, None))
    
    # Nếu không có XML tags, tokenize toàn bộ text
    if not segments:
        segments = [(text_with_xml, None)]
    
    # Tokenize từng segment và gán labels
    # Track previous label để tránh 2 B- liên tiếp cho cùng entity
    prev_label = None
    
    for segment_text, entity_label in segments:
        segment_tokens = tokenizer.tokenize(segment_text)
        
        if entity_label:
            # Entity segment: B-label chỉ cho token đầu tiên, I-label cho tất cả token còn lại
            for i, token in enumerate(segment_tokens):
                tokens.append(token)
                
                if i == 0:
                    # Token đầu tiên của entity: B-label chỉ khi token trước không phải I- hoặc B- của cùng entity
                    if prev_label == f"I-{entity_label}" or prev_label == f"B-{entity_label}":
                        # Nếu token trước là I- hoặc B- của cùng entity, thì token này cũng là I-
                        # (tránh 2 B- liên tiếp cho cùng entity)
                        labels.append(f"I-{entity_label}")
                        prev_label = f"I-{entity_label}"
                    else:
                        # Token đầu tiên của entity mới: B-label
                        labels.append(f"B-{entity_label}")
                        prev_label = f"B-{entity_label}"
                else:
                    # Tất cả token còn lại: I-label (kể cả các word không phải subword)
                    labels.append(f"I-{entity_label}")
                    prev_label = f"I-{entity_label}"
        else:
            # Non-entity segment: O labels
            for token in segment_tokens:
                tokens.append(token)
                labels.append('O')
                prev_label = 'O'
    
    return source_text, target_text, tokens, labels


## 6. Test hàm với ví dụ


In [42]:
# Test với một ví dụ
if len(df) > 0:
    test_text = df['target_text_vi'].iloc[0]
    print(f"Text gốc:")
    print(f"  {test_text}")
    print()
    
    source, target, tokens, labels = demask_and_tokenize(test_text, pii_data, tokenizer)
    
    print(f"Source text (đã demask, không có XML tags):")
    print(f"  {source[:200]}...")
    print()
    
    print(f"Tokens ({len(tokens)} tokens):")
    print(f"  {tokens[:]}...")
    print()
    
    print(f"Labels ({len(labels)} labels):")
    print(f"  {labels[:]}...")
    print()
    
    # Đếm entities
    entity_count = len([l for l in labels if l != 'O'])
    unique_entities = set([l.split('-')[1] for l in labels if '-' in l])
    print(f"Entities tìm thấy: {sorted(unique_entities)}")
    print(f"Số entity tokens: {entity_count}/{len(tokens)}")


KeyError: 'target_text_vi'

## 7. Áp dụng cho toàn bộ dataset


In [ ]:
# Tạo các cột mới
df['source_text'] = ''
df['target_text'] = df['target_text_vi']
df['tokens'] = None
df['labels'] = None

print(f"Đang xử lý {len(df)} dòng...")
errors = 0

for idx in range(len(df)):
    if (idx + 1) % 1000 == 0:
        print(f"  Đã xử lý {idx + 1}/{len(df)} dòng...")
    
    try:
        text = df.loc[idx, 'target_text_vi']
        source, target, tokens, labels = demask_and_tokenize(text, pii_data, tokenizer)
        
        df.loc[idx, 'source_text'] = source
        df.loc[idx, 'target_text'] = target
        df.loc[idx, 'tokens'] = str(tokens)  # Lưu dưới dạng string
        df.loc[idx, 'labels'] = str(labels)  # Lưu dưới dạng string
    except Exception as e:
        errors += 1
        if errors <= 5:
            print(f"  ⚠ Lỗi ở dòng {idx}: {e}")
        df.loc[idx, 'source_text'] = ''
        df.loc[idx, 'target_text'] = text
        df.loc[idx, 'tokens'] = str([])
        df.loc[idx, 'labels'] = str([])

print(f"\n✓ Hoàn thành! Đã xử lý {len(df)} dòng")
if errors > 0:
    print(f"  ⚠ Có {errors} lỗi (đã bỏ qua)")


Đang xử lý 24296 dòng...
  Đã xử lý 1000/24296 dòng...
  Đã xử lý 2000/24296 dòng...
  Đã xử lý 3000/24296 dòng...
  Đã xử lý 4000/24296 dòng...
  Đã xử lý 5000/24296 dòng...
  Đã xử lý 6000/24296 dòng...
  Đã xử lý 7000/24296 dòng...
  Đã xử lý 8000/24296 dòng...
  Đã xử lý 9000/24296 dòng...
  Đã xử lý 10000/24296 dòng...
  Đã xử lý 11000/24296 dòng...
  Đã xử lý 12000/24296 dòng...
  Đã xử lý 13000/24296 dòng...
  Đã xử lý 14000/24296 dòng...
  Đã xử lý 15000/24296 dòng...
  Đã xử lý 16000/24296 dòng...
  Đã xử lý 17000/24296 dòng...
  Đã xử lý 18000/24296 dòng...
  Đã xử lý 19000/24296 dòng...
  Đã xử lý 20000/24296 dòng...
  Đã xử lý 21000/24296 dòng...
  Đã xử lý 22000/24296 dòng...
  Đã xử lý 23000/24296 dòng...
  Đã xử lý 24000/24296 dòng...

✓ Hoàn thành! Đã xử lý 24296 dòng


## 8. Chỉ giữ lại 4 cột cần thiết


In [ ]:
# Chỉ giữ lại 4 cột: source_text, target_text, tokens, labels
df_final = df[['source_text', 'target_text', 'tokens', 'labels']].copy()

print(f"✓ DataFrame cuối cùng có {len(df_final)} dòng và {len(df_final.columns)} cột")
print(f"\nCác cột:")
for col in df_final.columns:
    print(f"  - {col}")

# Kiểm tra dữ liệu
print(f"\nKiểm tra dữ liệu:")
print(f"  - Dòng có source_text: {df_final['source_text'].notna().sum()}/{len(df_final)}")
print(f"  - Dòng có target_text: {df_final['target_text'].notna().sum()}/{len(df_final)}")
print(f"  - Dòng có tokens: {df_final['tokens'].notna().sum()}/{len(df_final)}")
print(f"  - Dòng có labels: {df_final['labels'].notna().sum()}/{len(df_final)}")

# Hiển thị ví dụ
print(f"\n=== VÍ DỤ 3 DÒNG ĐẦU ===")
for i in range(min(3, len(df_final))):
    print(f"\nDòng {i}:")
    print(f"  source_text: {df_final['source_text'].iloc[i][:100]}...")
    print(f"  target_text: {df_final['target_text'].iloc[i][:100]}...")
    
    import ast
    try:
        tokens = ast.literal_eval(df_final['tokens'].iloc[i])
        labels = ast.literal_eval(df_final['labels'].iloc[i])
        print(f"  tokens: {len(tokens)} tokens")
        print(f"  labels: {len(labels)} labels")
        entity_count = len([l for l in labels if l != 'O'])
        print(f"  entities: {entity_count} tokens")
    except:
        pass


✓ DataFrame cuối cùng có 24296 dòng và 4 cột

Các cột:
  - source_text
  - target_text
  - tokens
  - labels

Kiểm tra dữ liệu:
  - Dòng có source_text: 24296/24296
  - Dòng có target_text: 24296/24296
  - Dòng có tokens: 24296/24296
  - Dòng có labels: 24296/24296

=== VÍ DỤ 3 DÒNG ĐẦU ===

Dòng 0:
  source_text: Cơ sở dữ liệu tiêm chủng nhân viên sẽ được duy trì bởi Thư ký. Tất cả các mục sẽ được xác minh bằng ...
  target_text: Cơ sở dữ liệu tiêm chủng nhân viên sẽ được duy trì bởi [JOBAREA]. Tất cả các mục sẽ được xác minh bằ...
  tokens: 62 tokens
  labels: 62 labels
  entities: 14 tokens

Dòng 1:
  source_text: Chào Ronny.Emard, buổi tư vấn chấn thương của bạn được lên lịch vào ngày 20.10.1999. Vui lòng đến cơ...
  target_text: Chào [USERNAME], buổi tư vấn chấn thương của bạn được lên lịch vào ngày [DATE]. Vui lòng đến cơ sở c...
  tokens: 40 tokens
  labels: 40 labels
  entities: 12 tokens

Dòng 2:
  source_text: Chào Hân, công ty chúng tôi đang lên kế hoạch gửi đi một dòng sản 

## 9. Lưu file kết quả


In [ ]:
# Lưu file CSV cuối cùng
output_file = 'data/dong10000_32000_d.csv'


df_final.to_csv(output_file, index=False, encoding='utf-8')
print(f"✓ Đã lưu file: {output_file}")
print(f"  Số dòng: {len(df_final)}")
print(f"  Số cột: {len(df_final.columns)}")
print(f"\nCấu trúc file:")
print(f"  - source_text: Text đã demask (không có XML tags)")
print(f"  - target_text: Text gốc với PII placeholders")
print(f"  - tokens: Danh sách tokens từ mBERT (dạng string)")
print(f"  - labels: Danh sách labels theo BIO format (dạng string)")


✓ Đã lưu file: data/dong10000_32000_d.csv
  Số dòng: 24296
  Số cột: 4

Cấu trúc file:
  - source_text: Text đã demask (không có XML tags)
  - target_text: Text gốc với PII placeholders
  - tokens: Danh sách tokens từ mBERT (dạng string)
  - labels: Danh sách labels theo BIO format (dạng string)
